## Convert Scraped Content to DataFrame
Convert the raw html scraped from the Selenium map scraper to an structured data set. Remove duplicate entries caused by clicking on the same municipality or clicking on an area outside a valid municipality. 

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def safe_scrape_file_read(file_path:str):
    """
    Safely reads the contents of a text file while it may be concurrently written to by another program.

    Parameters:
    - file_path (str): The path to the text file.

    Returns:
    - str: The contents of the text file.

    This function attempts to open the specified text file for reading, handling the case
    when it is being written to by another program by retrying until successful. It returns
    the contents of the file once successfully read.

    Example:
    >>> content = safe_scrape_file_read('path/to/your/file.txt')
    """
    while True:
        try:
            with open(file_path, 'r') as file:
                file_content = file.read()
            break
        except PermissionError:
            print("File is currently being written to. Retrying...")
    return file_content


In [3]:
## update filepath to desired scrape output
html_content = safe_scrape_file_read('20231119100453.txt')

soup = BeautifulSoup(html_content, 'html.parser')

panel_bodies = soup.find_all(class_='panel-body')

df = pd.DataFrame()

for panel_body in panel_bodies:
    panel_table = pd.DataFrame()
    tr_tags = panel_body.find_all('tr')
    for tr_tag in tr_tags:
        row = pd.DataFrame(([td.get_text(strip=True) for td in tr_tag.find_all('td')]))
        ## Set the tr table row labels as a column. 
        row = row.T.set_index(0).T
        panel_table = pd.concat([panel_table,row],axis=1)
    df = pd.concat([df,panel_table],axis=0)

In [4]:
## Raw output. need to remove duplicates next. 
df

,BauleitplÃ¤ne Bayern (Gemeinden im Landesportal),Hinweis zu den Daten:,Gemeinde/Stadt:,Bauleitplanungsseite:,Laufende Bauleitplanverfahren:,Abgeschlossene Bauleitplanverfahren:
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Denkendorf,Keine Angabe,https://www.gemeinde-denkendorf.de/leben/baule...,https://www.gemeinde-denkendorf.de/leben/bauge...
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Denkendorf,Keine Angabe,https://www.gemeinde-denkendorf.de/leben/baule...,https://www.gemeinde-denkendorf.de/leben/bauge...
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Denkendorf,Keine Angabe,https://www.gemeinde-denkendorf.de/leben/baule...,https://www.gemeinde-denkendorf.de/leben/bauge...
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Denkendorf,Keine Angabe,https://www.gemeinde-denkendorf.de/leben/baule...,https://www.gemeinde-denkendorf.de/leben/bauge...
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Denkendorf,Keine Angabe,https://www.gemeinde-denkendorf.de/leben/baule...,https://www.gemeinde-denkendorf.de/leben/bauge...
...,...,...,...,...,...,...
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Altmannstein,Keine Angabe,Keine Angabe,Keine Angabe
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Altmannstein,Keine Angabe,Keine Angabe,Keine Angabe
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Altmannstein,Keine Angabe,Keine Angabe,Keine Angabe
1,NaN,"FÃ¼r die Richtigkeit, AktualitÃ¤t und VollstÃ¤...",Altmannstein,Keine Angabe,Keine Angabe,Keine Angabe


In [13]:
# Show only those with a valid Bauleitplanungsseite
building_plans = df.loc[df['Bauleitplanungsseite:'].replace('Keine Angabe',np.nan).notna(),:].drop_duplicates().set_index('Gemeinde/Stadt:')
building_plans.to_csv('scrape_output.csv')

In [ ]:
building_plans